In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2,SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score,precision_score

# Importantation de mon fichier "Coeur"

In [2]:
data=pd.read_excel("Copie de Coeur.xlsx")
data


,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,homme,AA,140,289,0,Normal,172,Non,0.0,Ascendant,0
1,49,femme,DNA,160,180,0,Normal,156,Non,1.0,Plat,1
2,37,homme,AA,130,283,0,ST,98,Non,0.0,Ascendant,0
3,48,femme,ASY,138,214,0,Normal,108,Oui,1.5,Plat,1
4,54,homme,DNA,150,195,0,Normal,122,Non,0.0,Ascendant,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,homme,AT,110,264,0,Normal,132,Non,1.2,Plat,1
914,68,homme,ASY,144,193,1,Normal,141,Non,3.4,Plat,1
915,57,homme,ASY,130,131,0,Normal,115,Oui,1.2,Plat,1
916,57,femme,AA,130,236,0,LVH,174,Non,0.0,Plat,1


# faire une copie de ma dataframe 

In [3]:
df=data.copy()

In [4]:
df

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,homme,AA,140,289,0,Normal,172,Non,0.0,Ascendant,0
1,49,femme,DNA,160,180,0,Normal,156,Non,1.0,Plat,1
2,37,homme,AA,130,283,0,ST,98,Non,0.0,Ascendant,0
3,48,femme,ASY,138,214,0,Normal,108,Oui,1.5,Plat,1
4,54,homme,DNA,150,195,0,Normal,122,Non,0.0,Ascendant,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,homme,AT,110,264,0,Normal,132,Non,1.2,Plat,1
914,68,homme,ASY,144,193,1,Normal,141,Non,3.4,Plat,1
915,57,homme,ASY,130,131,0,Normal,115,Oui,1.2,Plat,1
916,57,femme,AA,130,236,0,LVH,174,Non,0.0,Plat,1


# Verification si il y a des doublons

In [5]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
913    False
914    False
915    False
916    False
917    False
Length: 918, dtype: bool

# Compter les doublons

In [6]:
df.duplicated().value_counts()

False    918
dtype: int64

In [7]:
#Puisque qu'il n'y a pas de doublons Verifions maintenant s'il existe des valueurs manquantes dans notre dataframe

In [8]:
df.isna().value_counts()

AGE    SEXE   TDT    PAR    CHOLESTEROL  GAJ    ECG    FCMAX  ANGINE  DEPRESSION   PENTE  CŒUR 
False  False  False  False  False        False  False  False  False   False        False  False    918
dtype: int64

In [9]:
#Il n'y a pas de bvaleur manquante

# Vérifier si pas de constante

In [10]:
df.nunique()

AGE             50
SEXE             2
TDT              4
PAR             67
CHOLESTEROL    222
GAJ              2
ECG              3
FCMAX          119
ANGINE           2
DEPRESSION      53
PENTE            3
CŒUR             2
dtype: int64

In [11]:
#Nous n'avons pas eu de constante dans notre dataframe
#tand que nous ne trouvons de modalite==1 

# Recodage des variables qualitatives

In [12]:
#D'abord voyant  l'objet de nos colonnes 

In [13]:
df.select_dtypes("object").columns

Index(['SEXE', 'TDT', 'ECG', 'ANGINE', 'PENTE'], dtype='object')

In [14]:
#La connaissance des categaries de ma dataframe.
#Par exemple je voudrais connaitre CATEGORIES de colonne SEXE

In [15]:
df['SEXE'].astype('category')

0      homme
1      femme
2      homme
3      femme
4      homme
       ...  
913    homme
914    homme
915    homme
916    femme
917    homme
Name: SEXE, Length: 918, dtype: category
Categories (2, object): ['femme', 'homme']

In [16]:
#Connaissant maintenat la CATEGORIE essayons maintenat de le recoder

# homme=1 et femme=0

In [17]:
df['SEXE'].astype('category').cat.codes

0      1
1      0
2      1
3      0
4      1
      ..
913    1
914    1
915    1
916    0
917    1
Length: 918, dtype: int8

In [18]:
#faisons de meme pour toutes les colonnes qui sont de  formes categorielles

In [19]:
#pour cela nous allons creer une foction qui va prendre en parametre df

#Puis nous allons utilisations qui va parcourir chaque colonne

#Pour toute colonne qui est de forme categorielle sera automatiquement recoder grace à "astype('category').cat.codes"

In [20]:
def recoder(df):
    for i in df.select_dtypes('object').columns:
        df[i]=df[i].astype('category').cat.codes
    return(df)
recoder(df)

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,1,0,140,289,0,1,172,0,0.0,0,0
1,49,0,3,160,180,0,1,156,0,1.0,2,1
2,37,1,0,130,283,0,2,98,0,0.0,0,0
3,48,0,1,138,214,0,1,108,1,1.5,2,1
4,54,1,3,150,195,0,1,122,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,1,2,110,264,0,1,132,0,1.2,2,1
914,68,1,1,144,193,1,1,141,0,3.4,2,1
915,57,1,1,130,131,0,1,115,1,1.2,2,1
916,57,0,0,130,236,0,0,174,0,0.0,2,1


In [21]:
#division de nos feature en variable qualitative et numerique
   

In [22]:
var_numer=df._get_numeric_data().columns
var_qual=list(set(df.columns) - set(var_numer))
var_numer

Index(['AGE', 'SEXE', 'TDT', 'PAR', 'CHOLESTEROL', 'GAJ', 'ECG', 'FCMAX',
       'ANGINE', 'DEPRESSION ', 'PENTE', 'CŒUR'],
      dtype='object')

In [23]:
var_qual

[]

# Normalisation des variables quantitaives

In [34]:
def normalis(df):
    for col in var_numer:
        df[col]=df[col]/df[col].max()
    return df
normalis(df)

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,1.0,0.000000,0.70,0.479270,0.0,0.5,0.851485,0.0,0.000000,0.0,0.0
1,0.636364,0.0,1.000000,0.80,0.298507,0.0,0.5,0.772277,0.0,0.161290,1.0,1.0
2,0.480519,1.0,0.000000,0.65,0.469320,0.0,1.0,0.485149,0.0,0.000000,0.0,0.0
3,0.623377,0.0,0.333333,0.69,0.354892,0.0,0.5,0.534653,1.0,0.241935,1.0,1.0
4,0.701299,1.0,1.000000,0.75,0.323383,0.0,0.5,0.603960,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,0.584416,1.0,0.666667,0.55,0.437811,0.0,0.5,0.653465,0.0,0.193548,1.0,1.0
914,0.883117,1.0,0.333333,0.72,0.320066,1.0,0.5,0.698020,0.0,0.548387,1.0,1.0
915,0.740260,1.0,0.333333,0.65,0.217247,0.0,0.5,0.569307,1.0,0.193548,1.0,1.0
916,0.740260,0.0,0.000000,0.65,0.391376,0.0,0.0,0.861386,0.0,0.000000,1.0,1.0


In [25]:
#Création d'un dataframe X qui va contenir "age"  en pente --a-- et "coeur" en --y--

In [26]:
# Division des données en train et test

In [27]:
# division de notre dataset en feature et target

In [35]:
X=df.drop("CŒUR",axis=1)

In [36]:
Y=df["CŒUR"]

In [37]:
X

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE
0,0.519481,1.0,0.000000,0.70,0.479270,0.0,0.5,0.851485,0.0,0.000000,0.0
1,0.636364,0.0,1.000000,0.80,0.298507,0.0,0.5,0.772277,0.0,0.161290,1.0
2,0.480519,1.0,0.000000,0.65,0.469320,0.0,1.0,0.485149,0.0,0.000000,0.0
3,0.623377,0.0,0.333333,0.69,0.354892,0.0,0.5,0.534653,1.0,0.241935,1.0
4,0.701299,1.0,1.000000,0.75,0.323383,0.0,0.5,0.603960,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
913,0.584416,1.0,0.666667,0.55,0.437811,0.0,0.5,0.653465,0.0,0.193548,1.0
914,0.883117,1.0,0.333333,0.72,0.320066,1.0,0.5,0.698020,0.0,0.548387,1.0
915,0.740260,1.0,0.333333,0.65,0.217247,0.0,0.5,0.569307,1.0,0.193548,1.0
916,0.740260,0.0,0.000000,0.65,0.391376,0.0,0.0,0.861386,0.0,0.000000,1.0


In [38]:
Y

0      0.0
1      1.0
2      0.0
3      1.0
4      0.0
      ... 
913    1.0
914    1.0
915    1.0
916    1.0
917    0.0
Name: CŒUR, Length: 918, dtype: float64

In [32]:
# repatition de nos données en données d'entrainement train et de test

In [39]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=5)

In [41]:
#les dimensions de nos jeux données repartie

In [42]:
print("les dimensions de X_train sont {}".format(X_train.shape))


les dimensions de X_train sont (642, 11)


In [43]:
print("les dimensions de X_test sont {}".format(X_test.shape))


les dimensions de X_test sont (276, 11)


In [44]:

print("les dimensions de Y_test sont {}".format(Y_test.shape))

les dimensions de Y_test sont (276,)


In [45]:
print("les dimensions de Y_train sont {}".format(Y_train.shape))


les dimensions de Y_train sont (642,)


# ENTRAINEMENT DU MODEL

In [46]:
#choisissons un modele et instation le

In [47]:
logisticRegr = LogisticRegression()

In [48]:
LogisticRegression()

LogisticRegression()

In [49]:
#Entrainons notre modele 

In [50]:
logisticRegr.fit(X_train, Y_train)

LogisticRegression()

In [51]:

#creons une variable "predication " qui va contenir la valeur predite de X_test


In [52]:
prediction=logisticRegr.predict(X_test)

In [53]:
#voyons enssemble le resultat

In [57]:
prediction.mean()

0.5869565217391305

In [59]:
prediction

array([1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
       0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
       1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1.,
       1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.,
       1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1.,
       1., 0., 1., 1., 1.

In [60]:
#apres avoir predict les differentes valeurs, etablissons maintenant le score de notre model dans une variable "Score"

In [61]:
score = logisticRegr.score(X_test, Y_test)

In [62]:
score

0.8659420289855072

In [63]:
#Le score tend vers 0 mais très proche de 1.

In [64]:
#Vraiment je ne sais pas trop comprendre par là

# EVALUATION Du MODEL

In [65]:
precision = precision_score(Y_test, prediction)
 

In [66]:
precision

0.8703703703703703

In [67]:
recall = recall_score(Y_test, prediction)

In [68]:
recall

0.8980891719745223

# CONFUSION MATRIX

In [69]:
# pour mieux comprendre notre model utilisons "la matrice de confusion"

In [70]:
predict_value=pd.Series(logisticRegr.predict(X_test),name="prediction")


In [73]:
df_confus = pd.crosstab(Y_test, predict_value)


In [74]:
df_confus

prediction,0.0,1.0
CŒUR,,
0.0,16,28
1.0,12,15
